In [2]:
import pandas as pd

In [3]:
index = pd.MultiIndex.from_tuples([('Pressure', 'Pressure Comparison'),                                  
                                   ('Coordinates', 'Missing Coordinates' )],
                                   names=['Tests', 'Names'])

index_def = ['Instrument deployment depth defined in OOI Asset Management system / maximum pressure calculated from data file after eliminating data outside of global ranges and outliers (3 standard deviations).',
             'Check the coordinates in the data file against expected coordinates: obs, time, lat, lon, pressure (for instruments not located on a surface buoy).']

In [4]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame({'Definition':index_def},index=index)

,,Definition
Tests,Names,
Pressure,Pressure Comparison,Instrument deployment depth defined in OOI Asset Management system / maximum pressure calculated from data file after eliminating data outside of global ranges and outliers (3 standard deviations).
Coordinates,Missing Coordinates,"Check the coordinates in the data file against expected coordinates: obs, time, lat, lon, pressure (for instruments not located on a surface buoy)."


<p style="color:green; font-size:1.1em;">Pressure Comparison:</p>
<p style="color:blue">Deploy Depth   (vs)   Max Pressure</p>

In [1]:
index = pd.MultiIndex.from_tuples([('Data Coverage', 'Deployment Days'),
                                   ('Data Coverage', 'File Days'),
                                   ('Data Coverage', 'Start Gap'),
                                   ('Data Coverage','End Gap'),
                                   ('Data Coverage','Timestamps'),
                                   ('Data Coverage','Sampling Rate'),
                                   ('Time Order', 'Ascending Order' ),
                                   ('Time Order', 'Unique Timestamps' ),
                                   ('Gap Identification', 'End Gap'),
                                   ('Gap Identification', 'Gaps Count'),
                                   ('Gap Identification', 'Gap Days')],
                                   names=['Tests', 'Names'])

index_def = ['Number of days the instrument was deployed.',
             'Number of days for which there is at least 1 timestamp available for the instrument.',
             'Number of missing days at the start of a deployment: comparison of the deployment start date to the data start date.',
             'Number of missing days at the end of a deployment: comparison of the deployment end date to the data end date.',
             'Number of timestamps in a data file.',
             'Sampling rates are calculated from the differences in timestamps. The most common sampling rate is that which occurs >50%.',
             'Timestamps in the file are unique .',
             'Timestamps in the file are in ascending order',
             'Number of missing days at the end of a deployment: comparison of the deployment end date to the data end date.',
             'Number of gaps within a data file (exclusive of missing data at the beginning and end of a deployment). Gap is defined as >1 day of missing data.',
             'Number of days of missing data within a data file (exclusive of missing data at the beginning and end of a deployment).']

NameError: name 'pd' is not defined

In [ ]:
pd.DataFrame({'Definition':index_def},index=index)

| Names           |      Are                                                                                    |
|-----------------|:-------------------------------------------------------------------------------------------:|
| Deployment Days | Number of days the instrument was deployed.                                                 |
| File Days       | Number of days for which there is at least 1 timestamp available for the instrument.        |
| Start Gap       | Number of missing days at the start of a deployment.                                        |
| End Gap         | Number of missing days at the end of a deployment.                                          |
| Timestamps      | Number of timestamps in a data file.                                                        |
| Sampling Rate   | Calculated from the differences in timestamps. The most common is that which occurs >50%.   |

In [ ]:
#     # check stream names
#     omit_word = ['_dcl', '_imodem', '_conc']   
#     ss_new = []
    
#     for s in ustreams:
#         wordi = []
#         for word in omit_word:
#             if word in s:
#                 wordi.append(word)
#                 break

#         if wordi:
#             fix = s.split(wordi[0])
#             if len(fix) == 2:
#                 ss_new.append(fix[0] + fix[1].split('_recovered')[0])
#         elif '_recovered' in s:
#             ss_new.append(s.split('_recovered')[0])

#         else:
#             ss_new.append(s)

In [ ]:
df =  pd.DataFrame()
df_sum =  pd.DataFrame()
for ii in range(len(d_info)): 
    index_i = d_info.index.values[ii]
    print(index_i)
    if len(d_info['methods'].values[ii]) == 2:
        print("comparing data values with matching timestamps")
        ds0 = xr.open_dataset(d_info['datasets'].values[ii][0])
        ds0 = ds0.swap_dims({'obs': 'time'})        
        ds0_sci_vars = cf.return_science_vars(d_info['streams'].values[ii][0])
        ds0_method = d_info['methods'].values[ii][0]
        
        ds1 = xr.open_dataset(d_info['datasets'].values[ii][1])
        ds1 = ds1.swap_dims({'obs': 'time'})        
        ds1_sci_vars = cf.return_science_vars(d_info['streams'].values[ii][1])
        ds1_method = d_info['methods'].values[ii][1]
        
        # define preferred method
        if reviewfiles['method'].values[ii] == ds0_method:
            preferred_method = 'ds0'
            preferred_stream_name = d_info['streams'].values[ii][0]
            
        elif reviewfiles['method'].values[ii] == ds1_method: 
            preferred_method = 'ds1'
            preferred_stream_name = d_info['streams'].values[ii][1]
         
        print('preferred_method: ', preferred_method, '(', preferred_stream_name, ')')
        # find where the variable long names are the same
        ds0names = cf.long_names(ds0, ds0_sci_vars)
        ds0names.rename(columns={'name': 'name_ds0'}, inplace=True)
        
        ds1names = cf.long_names(ds1, ds1_sci_vars)
        ds1names.rename(columns={'name': 'name_ds1'}, inplace=True)
        
        mapping = pd.merge(ds0names, ds1names, on='long_name', how='inner')
        print(mapping)
        df, missing_data_list, diff_gzero_list, var_list = cf.compare_datasets(df,
                                                        mapping, preferred_method,
                                              index_i, ds0, ds0_method, ds1, ds1_method)
    else:
        print(d_info['note'].values[ii])
    
    fd_test = cf.found_data_in_another_stream(missing_data_list)
 
    comparison_details = cf.found_data_in_another_stream_diff(fd_test, 
                                                              diff_gzero_list, var_list)
    
    df_sum0 = pd.DataFrame({'data_comparison': [fd_test],
                            'comparison_details': [comparison_details]}
                           , index= [d_info.index.values[ii]])
    df_sum = df_sum.append(df_sum0)